## Imports

In [1]:
import cudaq
from cudaq import spin

from typing import List

import numpy as np
import time

## Hamiltonian map

In [2]:
def get_hamiltonian(edges):
    """
    Get the Hamiltonian mapping for an arbitrary graph

    Parameters
    ----------
    edges : List[Tuple[int, int]]
        List of edges in the graph
    
    Returns
    -------
    hamiltonian : cudaq.Operator
        Hamiltonian operator
    """
    # avoid 0 term in the Hamiltonian
    hamiltonian = 0.5 * spin.z(edges[0][0]) * spin.z(edges[0][1])
    for u, v in edges[1:]:
        hamiltonian += 0.5 * spin.z(u) * spin.z(v)
    return hamiltonian

## Actual QAOA

In [5]:
graph = [(0,1), (0,2), (0,3), (0,4), (1,2), (1,4), (2,3), (2,4), (2,5), (3,5), (4,5)]
graph = [(0, 7), (0, 3), (0, 6), (7, 6), (7, 8), (3, 4), (3, 1), (4, 9), (4, 8), (1, 5), (1, 9), (5, 9), (5, 2), (6, 2), (2, 8)]
graph = [(3, 4), (3, 1), (3, 2), (4, 14), (4, 11), (8, 15), (8, 2), (8, 11), (15, 5), (15, 9), (2, 5), (5, 6), (1, 12), (1, 10), (12, 7), (12, 0), (13, 14), (13, 7), (13, 9), (14, 6), (6, 11), (7, 10), (10, 0), (9, 0)]

# graph = [(16, 20), (16, 25), (16, 13), (20, 7), (20, 10), (18, 23), (18, 8), (18, 0), (23, 9), (23, 10), (7, 24), (7, 27), (15, 27), (15, 5), (15, 22), (27, 17), (6, 24), (6, 1), (6, 17), (24, 11), (12, 19), (12, 25), (12, 26), (19, 2), (19, 28), (3, 10), (3, 13), (3, 21), (13, 4), (25, 0), (14, 22), (14, 9), (14, 21), (22, 28), (9, 5), (8, 2), (8, 29), (1, 17), (1, 4), (2, 5), (0, 26), (26, 29), (29, 28), (4, 11), (11, 21)]
# graph = [(16, 20), (16, 25), (16, 2), (20, 0), (20, 19), (15, 24), (15, 19), (15, 7), (24, 4), (24, 13), (21, 22), (21, 12), (21, 11), (22, 14), (22, 7), (3, 10), (3, 6), (3, 14), (10, 18), (10, 4), (4, 23), (14, 6), (8, 18), (8, 0), (8, 5), (18, 1), (0, 25), (2, 11), (2, 17), (11, 17), (1, 9), (1, 12), (9, 23), (9, 5), (17, 13), (12, 23), (13, 19), (6, 5), (25, 7)]
# graph = [(6, 18), (6, 17), (6, 19), (18, 13), (18, 0), (4, 15), (4, 8), (4, 2), (15, 10), (15, 16), (5, 16), (5, 19), (5, 11), (16, 10), (19, 17), (11, 14), (11, 12), (14, 8), (14, 1), (10, 2), (13, 17), (13, 7), (7, 9), (7, 1), (8, 3), (3, 12), (3, 2), (12, 9), (0, 1), (0, 9)]

# graph = [(4, 9), (4, 6), (4, 0), (9, 11), (9, 0), (3, 7), (3, 12), (3, 11), (7, 5), (7, 2), (6, 8), (6, 14), (5, 13), (5, 14), (13, 8), (13, 1), (11, 15), (10, 15), (10, 0), (10, 8), (15, 2), (1, 12), (1, 2), (12, 14)]


edges_1 = [graph[i][0] for i in range(len(graph))]
edges_2 = [graph[i][1] for i in range(len(graph))]
hamiltonian = get_hamiltonian(graph)

# Problem parameters
qubit_count: int = hamiltonian.get_qubit_count()
layer_count: int = 2
parameter_count: int = 2 * layer_count
shots_count: int = 1000

@cudaq.kernel
def kernel_qaoa(edges_src: List[int], edges_tgt: List[int], qubit_count: int, layer_count: int, thetas: List[float]):
    """
    QAOA ansatz for Max-Cut
    
    Parameters
    ----------
    edges : List[Tuple[int, int]]
        The edges of the graph.
    qubit_count : int  
        The number of qubits.
    layer_count : int
        The number of layers in the QAOA ansatz.
    thetas : List[float]
        The angles for the QAOA ansatz.
    """
    qvector = cudaq.qvector(qubit_count)

    # Create superposition
    h(qvector)

    # Loop over the layers
    for layer in range(layer_count):
        for i, u in enumerate(edges_src):
            v = edges_tgt[i]
            x.ctrl(qvector[u], qvector[v])
            rz(2.0 * thetas[layer], qvector[v])
            x.ctrl(qvector[u], qvector[v])

        # Mixer unitary
        for qubit in range(qubit_count):
            rx(2.0 * thetas[layer + layer_count], qvector[qubit])

print(cudaq.draw(kernel_qaoa, edges_1, edges_2, qubit_count, layer_count, [1,1,1,1]))

# Make it repeatable with fixing random seeds
cudaq.set_random_seed(15)
np.random.seed(32)

# Specify the optimizer and its initial parameters for the angles in the layers
optimizer = cudaq.optimizers.COBYLA()
optimizer.initial_parameters = np.random.uniform(-np.pi / 8.0, np.pi / 8.0, parameter_count)
print("Initial parameters = ", optimizer.initial_parameters)

def objective(parameters):
    """
    Compute the expected value of the hamiltonian with respect to the kernel.

    Parameters
    ----------
    parameters : List[float]
        The parameters to optimize. Contains the angles for the qaoa ansatz.

    Returns
    -------
    result : float
        The expectation value of the hamiltonian: `<state(params) | H | state(params)>`
    """
    return cudaq.observe(kernel_qaoa, hamiltonian, edges_1, edges_2, qubit_count, layer_count, parameters).expectation()

optimal_expectation, optimal_parameters = optimizer.optimize(
    dimensions=parameter_count, function=objective)

# Print the optimized value and its parameters
print("Optimal expectation value = ", optimal_expectation)
print("Optimal parameters = ", optimal_parameters)
# print("No of iter = ", len(result_time))

# Sample the circuit using the optimized parameters
counts = cudaq.sample(kernel_qaoa, edges_1, edges_2, qubit_count, layer_count, optimal_parameters, shots_count=1000000)
results = sorted(counts.items(), key=lambda x: x[1], reverse=True)
for key, value in results[:10]:
    print(f"{key}: {value}")



      ╭───╮                                                              »
 q0 : ┤ h ├──────────────────────────────────────────────────────────────»
      ├───┤                   ╭───╮╭───────╮╭───╮                        »
 q1 : ┤ h ├───────────────────┤ x ├┤ rz(0) ├┤ x ├────────────────────────»
      ├───┤                   ╰─┬─╯╰───────╯╰─┬─╯╭───╮╭───────╮╭───╮     »
 q2 : ┤ h ├─────────────────────┼─────────────┼──┤ x ├┤ rz(0) ├┤ x ├─────»
      ├───┤                     │             │  ╰─┬─╯╰───────╯╰─┬─╯     »
 q3 : ┤ h ├──●─────────────●────●─────────────●────●─────────────●───────»
      ├───┤╭─┴─╮╭───────╮╭─┴─╮                                           »
 q4 : ┤ h ├┤ x ├┤ rz(0) ├┤ x ├──●─────────────●────●─────────────●───────»
      ├───┤╰───╯╰───────╯╰───╯  │             │    │             │       »
 q5 : ┤ h ├─────────────────────┼─────────────┼────┼─────────────┼───────»
      ├───┤                     │             │    │             │       »
 q6 : ┤ h ├──────────────

In [4]:
x_vec = results[0][0]

cost = 0
for edge in graph:
    n1, n2 = edge
    print(edge)
    cost += int(x_vec[n1])*(1-int(x_vec[n2])) + int(x_vec[n2])*(1-int(x_vec[n1]))


cost

(3, 4)
(3, 1)
(3, 2)
(4, 14)
(4, 11)
(8, 15)
(8, 2)
(8, 11)
(15, 5)
(15, 9)
(2, 5)
(5, 6)
(1, 12)
(1, 10)
(12, 7)
(12, 0)
(13, 14)
(13, 7)
(13, 9)
(14, 6)
(6, 11)
(7, 10)
(10, 0)
(9, 0)


22